In [21]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [22]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 200

In [23]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("data.json", "rt") as INFILE:
    jsondata = json.load(INFILE)

In [24]:
def wrangle_data(rawdata):
    dic = jsondata['data'][0]
    males = dic['males']
    maledeaths = dic['maledeaths']
    females = dic['females']
    femaledeaths = dic['femaledeaths']
    ageranges=[x['age'] for x in males]
    def min_age(agerange):
        agerange=agerange.replace('+','')
        start=agerange.split('_')[0]
        return int(start)
    ageranges.sort(key=min_age)
    df=pd.DataFrame(index=ageranges, columns=['males','male deaths','females','female deaths','total','total deaths'])
    for entry in males:
        ageband=entry['age']
        df.loc[ageband, 'males']=entry['value']
    for entry in maledeaths:
        ageband=entry['age']
        df.loc[ageband, 'male deaths']=entry['value']
    for entry in females:
        ageband=entry['age']
        df.loc[ageband, 'females']=entry['value']
    for entry in femaledeaths:
        ageband=entry['age']
        df.loc[ageband, 'female deaths']=entry['value']
    df['total']=df['males']+df['females']
    df['total deaths']=df['male deaths']+df['female deaths']
    return df

In [25]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    filters = [
        'areaType=Nation',
        'areaName=England'
    ]
    structure={
        "males": "maleCases",
        "females": "femaleCases",
        "maledeaths": "maleDeaths28Days",
        "femaledeaths": "femaleDeaths28Days",
    }
    api=Cov19API(filters=filters,structure=structure)
    cases_deaths_gender=api.get_json()
    with open("data.json", "wt") as OUTF:
        json.dump(cases_deaths_gender, OUTF)

In [26]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Refresh', # you may want to change this...
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to retrieve latest data",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='download'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='info', description='Refresh', icon='download', style=ButtonStyle(), tooltip='Click to ret…

In [27]:
agecols=wdg.SelectMultiple(
    options=['males', 'male deaths', 'females', 'female deaths', 'total', 'total deaths'], # options available
    value=['males', 'females'], # initial value
    rows=3, # rows of the selection box
    description='Sex',
    disabled=False
)
def age_graph(graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        df.plot(kind='bar', y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
        plt.show() # important - graphs won't update properly if this is missing
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
output=wdg.interactive_output(age_graph, {'graphcolumns': agecols})

display(agecols, output)

SelectMultiple(description='Sex', index=(0, 2), options=('males', 'male deaths', 'females', 'female deaths', '…

Output()